In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
ls DATA

Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv


# Data Import

In [3]:
df = pd.read_csv('DATA/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv')
df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,245762.0,...,1292776.0,1288753.0,1269532.0,1243884.0,1211977.0,1197322.0,1185428.0,1179938.0,1175379.0,1173231.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,209547.0,...,487111.0,486300.0,486154.0,487283.0,488823.0,489789.0,489865.0,490118.0,491195.0,493022.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,230594.0,...,1080810.0,1099111.0,1117633.0,1130101.0,1129983.0,1138594.0,1143043.0,1147409.0,1149477.0,1155724.0


The dataframe is in the wide format. I want it to have a column per region and row per timestamp. I will also filter the location down to include only New York. First let's check if our region's are unique.

## Export Metadata
Let's separate out the region meta info so we can reference it later.

In [4]:
meta = df.iloc[:, 0:9]

In [5]:
#import os
#os.mkdir('PKL')

In [6]:
# saving meta data
with open('PKL/meta.pkl', 'wb') as fp:
    pickle.dump(meta, fp)

## Subset
Subsetting to only Queens for now.

In [7]:
queens = df[df.CountyName == 'Queens County']

In [8]:
meta_cols = list(df.columns[0:9])

In [9]:
meta_cols.remove('RegionName')

In [10]:
queens = queens.drop(meta_cols, axis = 1)

In [11]:
queens.shape

(54, 297)

In [12]:
queens.columns[queens.isnull().sum() != 0][-1]

'2003-08-31'

It seems like we have full data of all queens zipcode starting from 2003 September. Let's cap it at that.

In [13]:
queens = queens.dropna(axis = 1)

## Percentage Increase
We are trying to find the best neighborhood to invest in. We can approach this in different ways. I can predict the housing price for coming year then calculate the difference OR I can predict the percent increase for each time point. I'll try the percent increase first.

In [14]:
def calculate_percent_increase(x1, x2):
    return (x2-x1)/x1

In [15]:
queens_p = queens.copy()
for i in range(queens_p.shape[1]-1,1, -1):
    prior = queens_p.iloc[:, i-1]
    current = queens_p.iloc[:, i] 
    queens_p.iloc[:, i] = calculate_percent_increase(prior, current)

In [16]:
queens_p = queens_p.drop(['2003-09-30'], axis = 1)

In [17]:
queens_p.head(2)

,RegionName,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
20,11375,0.006411,0.009131,0.003931,0.006311,0.004354,0.006254,0.004558,0.006666,0.011679,...,-0.002352,0.003036,-0.001953,0.000245,-0.004263,-0.000569,-0.003133,0.001783,0.005330,0.006475
108,11377,0.000258,0.000237,0.009213,0.011472,0.011599,0.007563,0.007991,0.009957,0.015381,...,0.003102,0.004646,0.012626,0.011440,0.013656,0.012643,0.008985,0.005495,0.004898,0.008270


## Transpose
Now I'll change the format to the wide format.

In [18]:
queens_p = queens_p.set_index('RegionName').transpose()

## Fix Datetime
Let's convert the index to datetime.

In [19]:
queens_p.index = pd.DatetimeIndex(queens_p.index)

In [20]:
queens_p.index = queens_p.index.strftime('%m/%Y')

In [21]:
queens_p.columns.name = None

## Exporting 
Now let's export the dataframe.

In [22]:
with open('PKL/timeseries_queens_p.pkl', 'wb') as fp:
    pickle.dump(queens_p, fp)

In [29]:
queens_p.to_csv('DATA/timeseries_queens_p.csv', header=True)

In [28]:
queens_p.describe()

,11375,11377,11355,11385,11373,11372,11101,11368,11354,11374,...,11411,11426,11428,11693,11004,11416,11436,11366,11363,11430
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,...,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,0.003468,0.003980,0.003774,0.003103,0.003162,0.003814,0.005330,0.003914,0.003601,0.003539,...,0.002739,0.002886,0.002316,0.003081,0.001975,0.002549,0.003111,0.003453,0.001965,0.001992
std,0.006692,0.007717,0.006652,0.006627,0.007821,0.008569,0.007859,0.007697,0.006709,0.008520,...,0.007363,0.006858,0.007345,0.010314,0.007057,0.008369,0.008167,0.006498,0.006203,0.006596
min,-0.019908,-0.016257,-0.019908,-0.012859,-0.020079,-0.019625,-0.017273,-0.018935,-0.019349,-0.023471,...,-0.014668,-0.018406,-0.016882,-0.021074,-0.017144,-0.022085,-0.016185,-0.019452,-0.018754,-0.015132
25%,-0.000548,-0.001315,-0.000384,-0.001108,-0.001822,-0.002126,-0.000049,-0.000367,0.000622,-0.002761,...,-0.001037,-0.000602,-0.001912,-0.004706,-0.002318,-0.001192,-0.001953,-0.000805,-0.001898,-0.001924
50%,0.004490,0.004404,0.004049,0.003410,0.004533,0.005018,0.004979,0.004382,0.003751,0.004402,...,0.003207,0.003729,0.003402,0.003297,0.002206,0.003577,0.004353,0.004218,0.002372,0.002994
75%,0.007610,0.009318,0.008065,0.008352,0.008083,0.009495,0.009990,0.008675,0.008048,0.008854,...,0.007580,0.007032,0.006429,0.010596,0.006685,0.007881,0.008660,0.008273,0.005806,0.005940
max,0.019103,0.026217,0.019747,0.017497,0.020607,0.022414,0.025790,0.020306,0.020037,0.021030,...,0.021923,0.019790,0.020310,0.029752,0.018371,0.022386,0.021145,0.018679,0.018841,0.019948
